In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data1 = "D:\\1_kaoyan\college_ending\\400仪表数据--全\middata - 副本\仪器分时数据——按照不同类型类型来区分(单相,三相)\\1_有数据\处理后\核心数据\FourHundred_loadprofile_three_P8311三相表相1的平均电压.csv"
data2 = "D:\\1_kaoyan\college_ending\\400仪表数据--全\middata - 副本\仪器分时数据——按照不同类型类型来区分(单相,三相)\\1_有数据\处理后\核心数据\FourHundred_loadprofile_three_P8312三相表相2的平均电压.csv"
data3 = "D:\\1_kaoyan\college_ending\\400仪表数据--全\middata - 副本\仪器分时数据——按照不同类型类型来区分(单相,三相)\\1_有数据\处理后\核心数据\FourHundred_loadprofile_three_P8313三相表相3的平均电压.csv"

In [3]:

wrong_biao=[]
yichang_biao=[]
zhengchang_biao=[]

In [4]:
pd1 = pd.read_csv(data1,encoding='utf-8')
pd2=pd.read_csv(data2,encoding='utf-8')
pd3=pd.read_csv(data2,encoding='utf-8')

In [5]:
pd1.head(5)

,meterNo,10023160000011,10023160000012,10023160000013,10023160000015,10023160000016,10023160000019,10023160000020,10023160000023,10023160000024,...,10023160000341,10023160000342,10023160000343,10023160000345,10023160000346,10023160000347,10023160000349,10023160000350,10023160000352,10014164400011
0,2020/1/1 00:00,244.0,207.0,256.0,NaN,246.0,0.0,259.0,235.0,238.0,...,253.0,212.0,180.0,249.0,260.0,NaN,247.0,253.0,257.0,237.0
1,2020/1/1 00:15,246.0,207.0,254.0,NaN,246.0,0.0,259.0,234.0,237.0,...,254.0,214.0,180.0,249.0,260.0,NaN,247.0,253.0,258.0,237.0
2,2020/1/1 00:30,246.0,208.0,255.0,NaN,246.0,0.0,261.0,233.0,240.0,...,254.0,215.0,178.0,249.0,260.0,NaN,248.0,253.0,258.0,239.0
3,2020/1/1 00:45,246.0,207.0,252.0,NaN,246.0,0.0,261.0,236.0,242.0,...,253.0,216.0,178.0,249.0,261.0,NaN,247.0,253.0,258.0,238.0
4,2020/1/1 01:00,242.0,208.0,253.0,NaN,246.0,0.0,261.0,234.0,240.0,...,253.0,216.0,178.0,249.0,261.0,NaN,247.0,251.0,259.0,238.0


In [6]:
biao1=pd1.columns
biaotou=list(biao1)

In [7]:
for i in list(range(len(biaotou)))[1:]:
    
    #! 2、分离数据，把每一个仪器表单独提取出来
    str1=pd1[['meterNo',biaotou[i]]].copy(deep=1)
    str2=pd2[['meterNo',biaotou[i]]].copy(deep=1)
    str3=pd3[['meterNo',biaotou[i]]].copy(deep=1)

    # 2.1、把三相表和并在一起
    diyige=pd.concat([str1,str2,str3],axis=1)
    # 2.2、重新定义列名
    diyige.columns = ['meterNo',biaotou[i]+'1',
                    'MeterNo1',biaotou[i]+'2',
                    'MeterNo2',biaotou[i]+'3']
    # 2.3、把仪表名字，相1，相2，相3提出来
    diyige_right=diyige[['meterNo',biaotou[i]+'1',biaotou[i]+'2',biaotou[i]+'3']]

    # 2.4、填补空缺值
    diyige_right=diyige_right.fillna(0.0)
    print("2、分离数据成功\n")

    #! 3、计算0的占比
    # 3.1、当占比超过80%，输出数据一场
    num_0=diyige_right[biaotou[i]+'1'][diyige_right[biaotou[i]+'1']==0].count()
    num_all=diyige_right[biaotou[i]+'1'].count()
    bili_0=num_0/num_all
    
    if bili_0 >=0.8:
        print(biaotou[i]+"data error\n")
        # num_yichang=num_yichang+1
        wrong_biao.append(biaotou[i])
        continue
    
    print("3、计算0的占比成功\n")



    #! 4、计算不平衡度
    diyige_right['Mean_123']=1/3*(diyige_right.iloc[:,1]+diyige_right.iloc[:,2]+diyige_right.iloc[:,3])
    
    diyige_right['fen_zi']=abs((diyige_right.iloc[:,1]-diyige_right['Mean_123']))+abs((diyige_right.iloc[:,2]-diyige_right['Mean_123']))+abs((diyige_right.iloc[:,3]-diyige_right['Mean_123']))
    diyige_right['fen_mu']=3*diyige_right['Mean_123']
    diyige_right['不平衡度']=diyige_right['fen_zi']/(diyige_right['fen_mu']+0.001)

    print("4、计算不平衡度成功\n")

    # #! 5、开始画图
    # diyige_right['meterNo']=diyige_right['meterNo'].apply(pd.to_datetime)
    # plt.scatter(diyige_right['meterNo'],diyige_right['不平衡度'],color="g", s=3,label='不平衡度')
    # plt.show()
    # plt.savefig(fname=biaotou[i]+"pic.png",figsize=[10,10])
    # print("5、开始画图成功\n")

    #! 6、计算指标
    aver=diyige_right['不平衡度'].mean()
    jicha = diyige_right['不平衡度'].max()-diyige_right['不平衡度'].min()
    four_point = diyige_right['不平衡度'].quantile(0.75)
    
    if aver <= 0.2 and four_point<=0.2:
        zhengchang_biao.append(biaotou[i])
    else:
        yichang_biao.append(biaotou[i])
    
    print(biaotou[i],aver,jicha)
    print("6、计算指标成功\n")

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000011  0.030975861285060302 1.333327823714227
6、计算指标成功

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000012  0.0003128286106941103 0.005098779288503628
6、计算指标成功

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000013  0.020271740048222686 1.2276010531524824
6、计算指标成功

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000015  0.03837874024075335 0.4280540244670475
6、计算指标成功

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000016  9.714593784651316e-05 0.002070390159849663
6、计算指标成功

2、分离数据成功

10023160000019 data error

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000020  0.030844258915382807 1.3333277074780274
6、计算指标成功

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000023  0.012647654673581503 1.333327300178129
6、计算指标成功

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000024  0.018328296818951154 0.3018453281563602
6、计算指标成功

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000027  0.015559904517279689 0.44597624256969676
6、计算指标成功

2、分离数据成功

3、计算0的占比成功

4、计算不平衡度成功

10023160000028  0.0203113

In [8]:
print('一共有多少个数据表'+str(len(biaotou))+'\n')
print('一共有多少个错误数据'+str(len(wrong_biao))+'\n')  
print(wrong_biao)
print('一共有多少正常数据'+str(len(zhengchang_biao))+'\n')
print(zhengchang_biao)
print('一共有多少异常数据'+str(len(yichang_biao))+'\n')
print(yichang_biao)
print('错误数据占比'+str(len(wrong_biao)/len(biaotou))+'\n')
print('异常数据占比'+str(len(yichang_biao)/len(biaotou))+'\n')
print('正常数据占比'+str(len(zhengchang_biao)/len(biaotou))+'\n')

一共有多少个数据表201

一共有多少个错误数据31

['10023160000019 ', '10023160000029 ', '10023160000042 ', '10023160000050 ', '10023160000067 ', '10023160000077 ', '10023160000079 ', '10023160000097 ', '10023160000101 ', '10023160000103 ', '10023160000109 ', '10023160000114 ', '10023160000129 ', '10023160000133 ', '10023160000166 ', '10023160000180 ', '10023160000185 ', '10023160000200 ', '10023160000228 ', '10023160000230 ', '10023160000234 ', '10023160000241 ', '10023160000249 ', '10023160000254 ', '10023160000271 ', '10023160000272 ', '10023160000282 ', '10023160000283 ', '10023160000289 ', '10023160000334 ', '10023160000347 ']
一共有多少正常数据158

['10023160000011 ', '10023160000012 ', '10023160000013 ', '10023160000015 ', '10023160000016 ', '10023160000020 ', '10023160000023 ', '10023160000024 ', '10023160000027 ', '10023160000028 ', '10023160000030 ', '10023160000032 ', '10023160000036 ', '10023160000037 ', '10023160000040 ', '10023160000041 ', '10023160000043 ', '10023160000044 ', '10023160000048 ', '10023

In [11]:
str(int('10023160000349 '))

'10023160000349'

In [13]:
str(int('10023160000349 '))+' '=='10023160000349 '

True

In [9]:
diyige_right['不平衡度'].describe()

count    35136.000000
mean         0.012113
std          0.020222
min          0.000000
25%          0.001961
50%          0.007246
75%          0.016010
max          0.651995
Name: 不平衡度, dtype: float64